In [1]:
from __future__ import print_function

import argparse
import math
import pickle
import random
import datetime
import numpy as np

import gym
import gym_2048

from q_learning import QLearning
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

/Users/yogesh/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
env = gym.make("2048-v0")
env.reset()
env.seed(1)

RENDER_ENV = False
TRAINING_ON = True
EPISODES = 10
MODEL_PATH = "outputs/keras-models/2048_model.h5"

board_size = int(math.sqrt(env.observation_space.shape[0]))
n_output = env.action_space.n

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: Environment '<class 'gym_2048.envs.game2048_env.Game2048Env'>' has deprecated methods. Compatibility code invoked.


In [3]:
QL = QLearning (
    n_x=board_size,
    n_y=n_output,
    save_path = MODEL_PATH,
    total_episodes=EPISODES,
    restore_model=True,
    is_training_on=TRAINING_ON
)

In [4]:
for episode in range(EPISODES):
    observation = env.reset()
    
    QL.curr_episode = episode
    
    while True:
        if RENDER_ENV: env.render()
            
        valid_move = False    
        action = None
        
        while not valid_move:
            
            # Choose an action based on observation
            if action == None: action = QL.choose_action(observation)
            
            observation_, reward, done, info = env.step(action)
            valid_move = info['valid']
            
            reward = QL.calculate_reward(valid_move, done, reward, observation_)
        
            QL.save_experience(observation=observation, action=action, 
                               reward=reward, observation_=observation_, is_game_over=done, is_move_valid=valid_move)
            
            action = (action + 1) % QL.n_y
        
        features, labels = QL.sample_from_experience()
        QL.train_model(features=features, labels=labels)
        
        if done:
            highest_tile_value = QL.get_highest_tile_value(observation_)
            QL.episodic_highest_tiles_track.append(highest_tile_value)
            print("Episode #", (episode + 1), " : Highest Tile: ", highest_tile_value)
            env.render()
            QL.plot_progress(y_data=QL.episodic_highest_tiles_track, y_label="Highest Tile Value", n_episode=episode)
            break
        

Episode # 1  : Highest Tile:  128
Score: 988.0
Highest: 128
[[  2   4   8   2]
 [  4   8  16 128]
 [  8  32   8  16]
 [  2   8   2   4]]
Episode # 2  : Highest Tile:  128
Score: 1104.0
Highest: 128
[[  2   8   4   8]
 [  4  16 128  32]
 [  8  32  16   4]
 [  4   2   8   2]]
Episode # 3  : Highest Tile:  64
Score: 700.0
Highest: 64
[[ 4 16  4  2]
 [16  8 32  4]
 [ 8 64 16 32]
 [ 2  8  4  2]]
Episode # 4  : Highest Tile:  128
Score: 1016.0
Highest: 128
[[  4   2  32   4]
 [ 16   8   4   2]
 [  8  16 128  16]
 [  2   4   8   4]]
Episode # 5  : Highest Tile:  256
Score: 2276.0
Highest: 256
[[  4   2  16   2]
 [  8  32   8   4]
 [  2  16 256   8]
 [ 64   2   4   2]]
Episode # 6  : Highest Tile:  128
Score: 1552.0
Highest: 128
[[  2   4  16   8]
 [128   2 128   2]
 [  8  16   2   8]
 [  2   4   8   4]]
Episode # 7  : Highest Tile:  128
Score: 1332.0
Highest: 128
[[  4   8   4   2]
 [128  32   2   4]
 [  4  64  32   8]
 [  2   4   8   4]]


KeyboardInterrupt: 